<a href="https://colab.research.google.com/github/AG-01/Paraphrase/blob/main/Paraphrase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install nltk


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install gingerit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from transformers import AutoTokenizer, T5ForConditionalGeneration, T5Tokenizer
from gingerit.gingerit import GingerIt
from joblib import Parallel, delayed
import multiprocessing
import functools
import torch

In [ ]:
nltk.download('punkt')

tokenizer = AutoTokenizer.from_pretrained("t5-large")
model = T5ForConditionalGeneration.from_pretrained('t5-large').to('cuda')
parser = GingerIt()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
def tokenize_paragraph(paragraph):
    words = word_tokenize(paragraph)
    return words

In [ ]:
def generate_paraphrase(sentence):
    input_text = 'paraphrase: ' + sentence + ' </s>'
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to('cuda')
    output = model.generate(input_ids=input_ids, max_length=1000, num_return_sequences=1, num_beams=5)
    paraphrased = tokenizer.decode(output[0], skip_special_tokens=True)
    grammar_corrected = parser.parse(paraphrased)['result']
    return grammar_corrected


In [ ]:
def paraphrase(paragraph):
    sentences = nltk.sent_tokenize(paragraph)
    num_cores = multiprocessing.cpu_count()
    paraphrases = Parallel(n_jobs=num_cores)(delayed(generate_paraphrase)(sentence) for sentence in sentences)
    return " ".join(paraphrases)


In [ ]:
user_input = input("Enter a paragraph to paraphrase: ")
paraphrased = paraphrase(user_input)
print("Paraphrased paragraph:")
print(paraphrased)

Enter a paragraph to paraphrase: kindly paraphrase this text as this is a test
Paraphrased paragraph:
Paraphrase: kindly paraphrase this text  this is a test.
